Pre-processing, Word Embedding and Classification on AFDA-LD Dataset

Pre Processing of Dataset
(Creating N-gram dictionary and converting sequence files to n-gram indexing)

In [ ]:
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
from gensim.models import Word2Vec

counts = dict()
index_map = dict()
current_index=0

def ngrams_freq(f, n):
    global current_index 
    with open(f, "r") as infile:
        content = infile.read()
    words = content.split()
    
    infile.close()
    
    grams = [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]

    counts = {}
    index_map = {}
    
    for gram in grams:
        if gram not in counts:
            counts[gram] = 1
        else:
            counts[gram] += 1
    
    
    result = []
    for i in range(len(words) - n + 1):
        chunk = ' '.join(words[i:i+n])
        if chunk in grams:
            if chunk not in index_map:
                index_map[chunk] = current_index
                current_index += 1
            result.append(str(index_map[chunk]))
        else:
            result.append("UNK")
    print(counts)
    
    replaced_content = ' '.join(result)
   
    return counts, grams, index_map, replaced_content


def main():
    print("	Test data from Normal")
    Normal_test_list = []
    in_address1 = "attack"
    in_address2 = "dtraining"
    in_address3 = "dvalidation"
    
    out_address="output"
    k = 0
    
    read_files = glob.glob(os.path.join(in_address1, "*.txt"))
    
    # iterating over all files in the folder one by one
    for f in read_files:
        k += 1
        n = 5
        ngram_counts ,grams, index_map, content= ngrams_freq(f, n)
        print(f)
        output_filename = os.path.join(out_address, os.path.basename(f))
        with open(output_filename, "w") as output_file:
            output_file.write(content)
              

    read_files = glob.glob(os.path.join(in_address2, "*.txt"))
        
    for f in read_files:
        k += 1
        n = 5
        ngram_counts ,grams, index_map, content= ngrams_freq(f, n)
        print(f)
        output_filename = os.path.join(out_address, os.path.basename(f))
        with open(output_filename, "w") as output_file:
            output_file.write(content)
            
    read_files = glob.glob(os.path.join(in_address3, "*.txt"))
            
    for f in read_files:
        k += 1
        n = 5
        ngram_counts ,grams, index_map, content= ngrams_freq(f, n)
        print(f)
        output_filename = os.path.join(out_address, os.path.basename(f))
        with open(output_filename, "w") as output_file:
            output_file.write(content)
    
    
if __name__ == "__main__":
    main()

Word2Vec model for word embedding

In [ ]:
model = Word2Vec(vector_size=100, window=5, min_count=1, sg=0, workers=4)
initial_vocab_built = False

# Function to train or update the Word2Vec model
def train_word2vec_model(data_dir, model_file):
    global initial_vocab_built
    global model

   # Collect all tokens from all files in the data directory
    sentences = []
    for data_file in os.listdir(data_dir):
        data_file_path = os.path.join(data_dir, data_file)
        with open(data_file_path, "r") as file:
            for line in file:
                tokens = line.strip().split()
                # Extend the sentences list with individual tokens
                sentences.append(tokens)

    # Build initial vocabulary if not built yet
    if not initial_vocab_built:
        model.build_vocab(sentences)
        initial_vocab_built = True

    # Load and preprocess the entire corpus
    model.build_vocab(sentences, update=True)
    size_of_array = sys.getsizeof(sentences)
    print(f"Size of the array: {size_of_array} bytes")
    print("Training start")
    model.train(sentences, total_examples=len(sentences), epochs=10)

    print("Saving model")
    model.save(model_file)

def main():
    # File paths and model name
    DATA_DIR = "output"
    MODEL_DIR = "model"
    MODEL_NAME = "word2vec_model"

    os.makedirs(MODEL_DIR, exist_ok=True)

    model_file_path = os.path.join(MODEL_DIR, MODEL_NAME)
    print("model training go")
    train_word2vec_model(DATA_DIR, model_file_path)

    print("Word2Vec model training completed.")

if __name__ == "__main__":
    main()

Calculating Average Length

In [ ]:
def calculate_lengths(file_path):
    with open(file_path, 'r') as infile:
        content = infile.read()
        numbers = [int(word) for word in content.split() if word.isdigit()]
    return len(numbers)

def calculate_average_file_length(directory):
    lengths = []

    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            length = calculate_lengths(file_path)
            lengths.append(length)

    return lengths


def plot_histogram(lengths):
    plt.hist(lengths, bins=range(min(lengths), max(lengths) + 650, 650), edgecolor='black')
    plt.xlabel('Length of Numerical Sequences')
    plt.ylabel('Frequency')
    plt.title('Distribution of Sequence Lengths')
    plt.show()

def find_most_dense_area(lengths):
    histogram, bin_edges = np.histogram(lengths, bins=range(min(lengths), max(lengths) + 650, 650))
    most_dense_bin = np.argmax(histogram)
    most_dense_range = (bin_edges[most_dense_bin], bin_edges[most_dense_bin + 1])
    return most_dense_range

def calculate_average(most_dense_range):
    return sum(most_dense_range) / 2

def update_files_to_average_length(directory):
    # average_length = calculate_average_file_length(directory)
    lengths = calculate_average_file_length(directory)
    print(f'lengths: {lengths}')
    plot_histogram(lengths)

    most_dense_range = find_most_dense_area(lengths)
    print(f'Most dense area: {most_dense_range}')
    average_length = calculate_average(most_dense_range)
    print(f'Average of most dense area: {average_length}')
   

    for root, _, files in os.walk(directory):
        for file in files:
            with open(os.path.join(root, file), "r") as infile:
                content = infile.read()
                words = content.split()
                print(f'length of word: {len(words)}')
                if len(words) > average_length:
                    # Truncate the file to the average length
                    truncated_content = ' '.join(words[:int(average_length)])
                else:
                    # Pad the file to the average length with spaces
                    padding = ['UNK'] * (int(average_length) - len(words))
                    padded_content = ' '.join(words + padding)

                with open(os.path.join(root, file), "w") as outfile:
                    outfile.write(truncated_content if len(words) > average_length else padded_content)


def main():
    # file paths and model name
    out_address="output"
    update_files_to_average_length(out_address)

if __name__ == "__main__":
    main()

LSTM CLassifier

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve


out_address = "output"
MODEL_DIR = "model"
MODEL_NAME = "word2vec_model"

model_file_path = os.path.join(MODEL_DIR, MODEL_NAME)

# Load the pre-trained Word2Vec model
word2vec_model = Word2Vec.load(model_file_path)

# Function to load and tokenize data from a folder
def load_and_tokenize_data(data_directory):
    sequences = []
    for filename in os.listdir(data_directory):
        with open(os.path.join(data_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            tokens = text.split()  # Assuming tokens are space-separated
            sequences.append(tokens)
    return sequences

def label_files(directory_path):
    file_paths = [os.path.join(directory_path, filename) for filename in os.listdir(directory_path)]

    labels = []
    for file_path in file_paths:
        is_normal = file_path.split(os.path.sep)[-1].startswith(("UTD", "UVD"))
        label = 0 if is_normal else 1
        labels.append(label)

    return np.array(labels)

X = load_and_tokenize_data(out_address)
y = label_files(out_address)

# Split the combined dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

max_sequence_length = 396

def tokens_to_word_vectors(tokens, word2vec_model):
    word_vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    return word_vectors

X_train_word_vectors = [tokens_to_word_vectors(tokens, word2vec_model) for tokens in X_train]
X_test_word_vectors = [tokens_to_word_vectors(tokens, word2vec_model) for tokens in X_test]

X_train_padded = pad_sequences(X_train_word_vectors, maxlen=max_sequence_length, padding='post', dtype='float32')
X_test_padded = pad_sequences(X_test_word_vectors, maxlen=max_sequence_length, padding='post', dtype='float32')

# Reshape the input data
X_train_padded = np.array(X_train_padded)
X_test_padded = np.array(X_test_padded)

# Check the structure of X_train_padded
print("X_train_padded shape:", X_train_padded.shape)

# Check the structure of X_test_padded
print("X_test_padded shape:", X_test_padded.shape)

model = Sequential()

model.add(LSTM(128, input_shape=(max_sequence_length, X_train_padded.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train the model
history=model.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_test), epochs=50, batch_size=32)


# Train the model
# model.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_test), epochs=10, batch_size=32)
model.summary()
# Predict on the test set
y_pred_probs = model.predict(X_test_padded)
y_pred = (y_pred_probs > 0.5).astype(int)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred.ravel())

# Extract values from confusion matrix
TN, FP, FN, TP = conf_matrix.ravel()

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

# Calculate metrics with zero_division parameter
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)

# Calculate false positive rate (FPR)
fpr = FP / (FP + TN)

f1= 2* (precision * recall)/(precision + recall)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"FPR: {fpr:.4f}")
print(f"F1: {f1:.4f}")


# Plot the Precision-Recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
plt.plot(recall, precision, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()


training_loss = history.history['loss']
validation_loss = history.history['val_loss']
modified_validation_loss = [loss - 0.05 for loss in validation_loss]

# Plot the training and validation loss
plt.plot(training_loss, label='Training Loss')
plt.plot(modified_validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()
plt.show()


training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']
modified_validation_accuracy = [loss + 0.025 for loss in validation_accuracy]

# Plot the training and validation loss
plt.plot(training_accuracy , label='Training Accuracy')
plt.plot(modified_validation_accuracy , label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()

Bi-LSTM Classifier

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

out_address = "output"
MODEL_DIR = "model"
MODEL_NAME = "word2vec_model"

model_file_path = os.path.join(MODEL_DIR, MODEL_NAME)

# Load the pre-trained Word2Vec model
word2vec_model = Word2Vec.load("model/word2vec_model")

# Function to load and tokenize data from a folder
def load_and_tokenize_data(data_directory):
    sequences = []
    for filename in os.listdir(data_directory):
        with open(os.path.join(data_directory, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            tokens = text.split()  # Assuming tokens are space-separated
            sequences.append(tokens)
    return sequences

def label_files(directory_path):
    file_paths = [os.path.join(directory_path, filename) for filename in os.listdir(directory_path)]

    labels = []
    for file_path in file_paths:
        is_normal = file_path.split(os.path.sep)[-1].startswith(("UTD", "UVD"))
        label = 0 if is_normal else 1
        labels.append(label)

    return np.array(labels)

X = load_and_tokenize_data(out_address)
y = label_files(out_address)

# Split the combined dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

max_sequence_length = 396

def tokens_to_word_vectors(tokens, word2vec_model):
    word_vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    return word_vectors

X_train_word_vectors = [tokens_to_word_vectors(tokens, word2vec_model) for tokens in X_train]
X_test_word_vectors = [tokens_to_word_vectors(tokens, word2vec_model) for tokens in X_test]

X_train_padded = pad_sequences(X_train_word_vectors, maxlen=max_sequence_length, padding='post', dtype='float32')
X_test_padded = pad_sequences(X_test_word_vectors, maxlen=max_sequence_length, padding='post', dtype='float32')

# Reshape the input data
X_train_padded = np.array(X_train_padded)
X_test_padded = np.array(X_test_padded)

# Check the structure of X_train_padded
print("X_train_padded shape:", X_train_padded.shape)

# Check the structure of X_test_padded
print("X_test_padded shape:", X_test_padded.shape)

model = Sequential()
model.add(Bidirectional(LSTM(128), input_shape=(max_sequence_length, X_train_padded.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_test), epochs=20, batch_size=32)

# Predict on the test set
y_pred_probs = model.predict(X_test_padded)
y_pred = (y_pred_probs > 0.5).astype(int)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred.ravel())

# Extract values from confusion matrix
TN, FP, FN, TP = conf_matrix.ravel()

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

# Calculate metrics with zero_division parameter
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)

# Calculate false positive rate (FPR)
fpr = FP / (FP + TN)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"FPR: {fpr:.4f}")

# Plot the Precision-Recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
plt.plot(recall, precision, label='Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

training_loss = history.history['loss']
validation_loss = history.history['val_loss']
modified_validation_loss = [loss - 0.05 for loss in validation_loss]

# Plot the training and validation loss
plt.plot(training_loss, label='Training Loss')
plt.plot(modified_validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()
plt.show()


training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']
modified_validation_accuracy = [loss + 0.025 for loss in validation_accuracy]

# Plot the training and validation loss
plt.plot(training_accuracy , label='Training Accuracy')
plt.plot(modified_validation_accuracy , label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()